In [1]:
import torch
import pandas as pd
import numpy
from functools import partial
from pathlib import Path
from huggingface_hub import constants as hub_c
from datasets import Dataset
from peft import get_peft_model, LoraConfig, TaskType, PeftModel
from transformers import Trainer, TrainingArguments, AutoTokenizer, DataCollatorForLanguageModeling
from transformers.models.llama.modeling_llama import LlamaForCausalLM

from evals import evaluate, load_eval_dataset
from utils import DatasetArgs, get_dataset_args

assert torch.cuda.is_available(), "CUDA not available"
device = torch.device("cuda")

seed = 42
torch.manual_seed(seed)

model_id = "meta-llama/Llama-3.2-1B"
tokenizer = AutoTokenizer.from_pretrained(model_id)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

base_model = LlamaForCausalLM.from_pretrained(model_id, torch_dtype="float16")

dataset_id = "FPB"

### Dataset setup
Run the cell for eval as well

In [2]:
args = get_dataset_args(tokenizer, Path(hub_c.HF_HUB_CACHE))

### Dataset preprocessing

In [3]:
def train_preprocess(args: DatasetArgs, example: dict):
    # Create prompt and target text
    prompt_args = [example[key] for key in args.prompt_args[dataset_id]]
    prompt = args.prompt_templates[dataset_id].format(*prompt_args)

    target = args.id2labels[dataset_id][example["label"]]
    
    # tokenize text
    return tokenizer(prompt + target, truncation=False)

In [ ]:
dataset_path = args.paths[dataset_id]
df_dataset = pd.read_csv(dataset_path / "train.csv",
                         delimiter=args.del_mapping[dataset_id],
                         names=args.names_mapping[dataset_id])

preprocess_func = partial(train_preprocess, args)
dataset = (Dataset
            .from_pandas(df_dataset)
            .map(preprocess_func,
                 batched=False,
                 remove_columns=args.columns[dataset_id])
            .filter(lambda sample: len(sample["input_ids"]) <= args.max_length))

dataset = dataset.train_test_split(test_size=0.1, seed=seed)

## LoRA Setup

In [ ]:
peft_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    r=8,
    lora_alpha=16,
    lora_dropout=0.01,
    target_modules=["q_proj", "v_proj"]
)

peft_model = get_peft_model(base_model, peft_config)
peft_model.print_trainable_parameters()

trainable params: 5,636,096 || all params: 1,241,450,496 || trainable%: 0.4540


## Trainer setup

In [6]:
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=False
)

out_dir = Path(rf"D:/models/expert-Llama-3_2-1B-{dataset_id}-expert256")
training_args = TrainingArguments(
    output_dir=str(out_dir),
    num_train_epochs=12,
    per_device_train_batch_size=8,
    gradient_accumulation_steps=4,
    learning_rate=1e-3,
    weight_decay=0.01,
    warmup_steps=128,
    logging_steps=32,
    save_steps=128,
    save_strategy="steps",
    eval_steps=128,
    eval_strategy="steps",
)

trainer = Trainer(
    model=peft_model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    data_collator=data_collator,
)

In [7]:
trainer.train()

  0%|          | 0/1308 [00:00<?, ?it/s]

{'loss': 3.5022, 'grad_norm': 1.8773185014724731, 'learning_rate': 0.00025, 'epoch': 0.29}
{'loss': 2.1105, 'grad_norm': 1.111354112625122, 'learning_rate': 0.0005, 'epoch': 0.59}
{'loss': 1.8909, 'grad_norm': 0.83847975730896, 'learning_rate': 0.00075, 'epoch': 0.88}
{'loss': 1.8484, 'grad_norm': 0.9039217829704285, 'learning_rate': 0.001, 'epoch': 1.17}


  0%|          | 0/49 [00:00<?, ?it/s]

{'eval_loss': 1.8227686882019043, 'eval_runtime': 2.0506, 'eval_samples_per_second': 189.21, 'eval_steps_per_second': 23.895, 'epoch': 1.17}
{'loss': 1.8198, 'grad_norm': 0.7252324223518372, 'learning_rate': 0.0009728813559322034, 'epoch': 1.47}
{'loss': 1.7596, 'grad_norm': 0.9131329655647278, 'learning_rate': 0.0009457627118644068, 'epoch': 1.76}
{'loss': 1.7466, 'grad_norm': 0.8325889706611633, 'learning_rate': 0.0009186440677966102, 'epoch': 2.06}
{'loss': 1.6769, 'grad_norm': 0.9228724837303162, 'learning_rate': 0.0008915254237288136, 'epoch': 2.35}


  0%|          | 0/49 [00:00<?, ?it/s]

...

  0%|          | 0/49 [00:00<?, ?it/s]

{'eval_loss': 2.1924102306365967, 'eval_runtime': 2.0669, 'eval_samples_per_second': 187.716, 'eval_steps_per_second': 23.706, 'epoch': 10.57}
{'loss': 0.949, 'grad_norm': 1.5981618165969849, 'learning_rate': 0.00010508474576271186, 'epoch': 10.86}
{'loss': 0.9095, 'grad_norm': 1.636603832244873, 'learning_rate': 7.796610169491526e-05, 'epoch': 11.16}
{'loss': 0.8865, 'grad_norm': 1.5029820203781128, 'learning_rate': 5.084745762711865e-05, 'epoch': 11.45}
{'loss': 0.9041, 'grad_norm': 1.4532837867736816, 'learning_rate': 2.3728813559322036e-05, 'epoch': 11.74}


  0%|          | 0/49 [00:00<?, ?it/s]

{'eval_loss': 2.2749266624450684, 'eval_runtime': 2.2017, 'eval_samples_per_second': 176.227, 'eval_steps_per_second': 22.255, 'epoch': 11.74}
{'train_runtime': 662.2032, 'train_samples_per_second': 63.207, 'train_steps_per_second': 1.975, 'train_loss': 1.3838878829909391, 'epoch': 12.0}


TrainOutput(global_step=1308, training_loss=1.3838878829909391, metrics={'train_runtime': 662.2032, 'train_samples_per_second': 63.207, 'train_steps_per_second': 1.975, 'total_flos': 1.74494483226624e+16, 'train_loss': 1.3838878829909391, 'epoch': 12.0})

# Eval

In [ ]:
dataset_id = "FPB"
ckpt_path = Path(rf"D:/models/expert-Llama-3_2-1B-{dataset_id}-expert256") / "checkpoint-4620"

base_model = LlamaForCausalLM.from_pretrained(model_id, torch_dtype="float16").eval()
expert_model = PeftModel.from_pretrained(base_model, ckpt_path, torch_dtype="float16").eval().to(device)

In [8]:
testset = load_eval_dataset(tokenizer, dataset_id, args)
results = evaluate(peft_model, tokenizer,
                   testset,
                   guidance=True,
                   token_opts=args.token_opts[dataset_id])

Loading FPB dataset from AdaptLLM/finance-tasks


60.21: 100%|██████████| 970/970 [00:37<00:00, 25.57it/s]
